In [1]:
#imports
import pandas as pd
import numpy as np
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
# Set Chrome options (optional)
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run browser in headless mode

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)
#using selenium to launch and scroll through the Google Jobs page
url = "https://www.google.com/search?q=data+science+jobs+united+states+in+the+last+1+week"
driver.get(url)


In [2]:
from bs4 import BeautifulSoup
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')
page_text = soup.get_text()

In [3]:
jobsDiv = soup.find('div', attrs = {'class':'crJ18e'}).find_all("div")

In [4]:
def get_jobs_url(jobsDiv):
    for links in jobsDiv[1:]:
        if links.getText() == "Jobs":
            if len(links.find_all('a', href=True)) == 0:
                pass
            else:
                split_str = (str(links.find_all("a")[0]).split(" "))
                for st in split_str:
                    if "href" in st:
                        url = "https://www.google.com"+st.split('"')[1].replace("amp;","")
                        return url
                    else:
                        pass
    return ""

In [5]:
# driver.quit()
url = get_jobs_url(jobsDiv)
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
print(url)

https://www.google.com/search?sca_esv=c920b7292afa99a8&q=data+science+jobs+united+states+in+the+last+1+week&udm=8&fbs=AEQNm0BfLny2ggB-KDliWrXpEAi8buBZhx33_saU4_gDeS9HJ1FMt4o6FAuaS3jSM1omJ8EwXoWuoqGLQsBFTHZ_f8yEbiOtan8PymD4_A9PfQ0C65BYneqQWVijFUoX3pO5_764U4nWZSvqAE_SDHyX-R7zo0LvqFJZLlG3xNQVFh-CtFlYITBYNGOWZuTY0tudX3akSdswNjiLUN4cPIJy9ry4wRGiqg&sa=X&ved=2ahUKEwjSiLyHuvWIAxVaEFkFHfQmGjoQs6gLegQIJhAB


In [6]:
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')
div = soup.find_all('div', attrs = {'class':'EimVGf'})
len(div)

10

In [7]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10)

In [8]:
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')
job_detail_div = soup.find_all('div', attrs = {'class':'EimVGf'})
len(div)

10

In [9]:
details_link = []
for div in job_detail_div:
    details_link.append(str(div.find_all("a")[0]).split(" ")[2].split('"')[1].replace("amp;",""))

In [147]:
job_details_dict = {"job_header":[],"job_sites":[],'Job highlights':[],'Qualifications':[],'Benefits':[],'Responsibilities':[],'Job description':[]}
for idx,link in enumerate(details_link[:10]):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(link)
    time.sleep(5)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    job_header = soup.find('div', attrs = {'class':'JmvMcb'}).get_text(separator="@@@###@@@").split("@@@###@@@")
    print("{}) {}".format(idx,job_header[0]))
    job_sites = soup.find('div', attrs = {'class':'gwBWYe'}).get_text(separator="@@@###@@@").split("@@@###@@@")
    job_details_header = soup.find('div', attrs = {'class':'NgUYpe'})
    job_details_header = [i.get_text() for i in job_details_header.find_all(["h1","h2","h3","h4","h5"])]
    job_details_text = soup.find('div', attrs = {'class':'NgUYpe'}).get_text(separator="@@@###@@@").replace("@@@###@@@","\n")
    for i in job_details_header:
        job_details_text = job_details_text.replace(i,"<###HEADER###>",1)
    job_details_dict["job_header"].append(job_header)
    job_details_dict["job_sites"].append(job_sites)
    sep = job_details_text.split("<###HEADER###>")[1:]
    for s,j in zip(sep,job_details_header):
        if j in list(job_details_dict.keys()):
            job_details_dict[j].append(s)
        # else:
        #     job_details_dict[j] = [s]
driver.quit()

0) Schaumburg Data Science Tutor
1) Schaumburg Data Science Tutor
2) Data Scientist- Direct Hire Authority
3) (USA) Principal, Data Scientist
4) Data Science Co Op
5) Interdisciplinary Biologist / Physical Scientist (Data Scientist)
6) Data Science Co Op
7) Data Science Co Op
8) Data Scientist
9) (USA) Principal, Data Scientist


In [143]:
job_details_header

['Job highlights',
 'Qualifications',
 'Benefits',
 'Responsibilities',
 'Job description']

In [145]:
job_details_dict

{'job_header': [['Schaumburg Data Science Tutor',
   'Varsity Tutors, a Nerdy Company • Schaumburg, IL •  via LinkedIn',
   '2 days ago',
   'Full-time',
   'No Degree Mentioned'],
  ['Mesa Data Science Tutor',
   'Varsity Tutors, a Nerdy Company • Mesa, AZ •  via LinkedIn',
   '2 days ago',
   'Full-time',
   'No Degree Mentioned'],
  ['Data Analyst Asc - Level 1',
   'Lockheed Martin • Fort Worth, TX •  via Lockheed Martin',
   '11 days ago',
   'Full-time and Part-time',
   'No Degree Mentioned',
   'Paid time off',
   'Health insurance'],
  ['(USA) Principal, Data Scientist',
   'Walmart • Springdale, AR •  via ZipRecruiter',
   '110K–220K a year',
   'Full-time and Part-time',
   'Dental insurance',
   'Paid time off',
   'Health insurance'],
  ['Data Science Internship (Remote Eligible) - Graduate Student Preferred',
   "Lands' End Japan • Worcester, MA •  via LinkedIn",
   '1 day ago',
   'Internship'],
  ['Data Science Internship (Remote Eligible) - Graduate Student Preferred',

In [148]:
pd.DataFrame(job_details_dict).to_csv("jobs.csv")
pd.DataFrame(job_details_dict)

,job_header,job_sites,Job highlights,Qualifications,Benefits,Responsibilities,Job description
0,"[Schaumburg Data Science Tutor, Varsity Tutors...",[Apply on LinkedIn ],\nIdentified by Google from the original job p...,\nYou have excellent communication skills and ...,\nEnjoy competitive rates and get paid 2x per ...,\n“work remotely”) using our purpose-built Liv...,\nSchaumburg Data Science Tutor Jobs\nThe Vars...
1,"[Schaumburg Data Science Tutor, Varsity Tutors...",[Apply on LinkedIn ],\nIdentified by Google from the original job p...,\nYou have excellent communication skills and ...,\nEnjoy competitive rates and get paid 2x per ...,\n“work remotely”) using our purpose-built Liv...,\nSchaumburg Data Science Tutor Jobs\nThe Vars...
2,"[Data Scientist- Direct Hire Authority, US Dep...","[Apply on Glassdoor , Apply on Washington Pos...",\nIdentified by Google from the original job p...,\nU.S. Citizenship is required\nIncumbent will...,\nA career with the U.S. government provides e...,"\nMakes recommendations on analytical, statist...",\nDuties\n• Makes recommendations on analytica...
3,"[(USA) Principal, Data Scientist, Walmart • Sp...","[Apply on ZipRecruiter , Apply on Jobcase , ...",\nIdentified by Google from the original job p...,\n7+ years of professional hands-on experience...,\nBenefits & Perks\nBeyond our great compensat...,\nYou will work closely with cross-functional ...,\nPosition Summary...\nWhat you'll do...\nAbou...
4,"[Data Science Co Op, Gorbel • New York • via ...","[Apply on LinkedIn , Apply on Monster , Appl...",\nIdentified by Google from the original job p...,"\nMust be currently enrolled in a full-time, d...",\nThe wage range for this role takes into acco...,"\nAs a Data Science Intern, you’ll have the op...",\nGorbel’s mission is simple: We improve peopl...
5,[Interdisciplinary Biologist / Physical Scient...,"[Apply on ZipRecruiter , Apply on Rise AI Job...",\nIdentified by Google from the original job p...,"\nFor current Federal employees, if hours work...",\nPart-time hours are prorated\nYou will not r...,\nFor periods of time that reflect military se...,\nThis position is located in Natural Resource...
6,"[Data Science Co Op, Gorbel • New York • via ...","[Apply on LinkedIn , Apply on Monster , Appl...",\nIdentified by Google from the original job p...,"\nMust be currently enrolled in a full-time, d...",\nThe wage range for this role takes into acco...,"\nAs a Data Science Intern, you’ll have the op...",\nGorbel’s mission is simple: We improve peopl...
7,"[Data Science Co Op, Gorbel • New York • via ...","[Apply on LinkedIn , Apply on Monster , Appl...",\nIdentified by Google from the original job p...,"\nMust be currently enrolled in a full-time, d...",\nThe wage range for this role takes into acco...,"\nAs a Data Science Intern, you’ll have the op...",\nGorbel’s mission is simple: We improve peopl...
8,"[Data Scientist, FIS Global • Bellevue, WA • ...","[Apply on Glassdoor , Apply on Talent.com , ...",\nIdentified by Google from the original job p...,\nExperienced (relevant combo of work and educ...,\nOpportunities to innovate in fintech\nTools ...,\nWe are looking for a Senior Data Scientist t...,\nPosition Type :\nFull time\nType Of Hire :\n...
9,"[(USA) Principal, Data Scientist, Walmart • Sp...","[Apply on ZipRecruiter , Apply on Jobcase , ...",\nIdentified by Google from the original job p...,\n7+ years of professional hands-on experience...,\nBenefits & Perks\nBeyond our great compensat...,\nYou will work closely with cross-functional ...,\nPosition Summary...\nWhat you'll do...\nAbou...


In [86]:

job_details_header

['Job_highlights',
 'Qualifications',
 'Benefits',
 'Responsibilities',
 'Job_description']

In [ ]:
all_tags = soup.find_all(["h1","h2","h3","h4","h5","ul","li","p"])

In [ ]:
for t in all_tags:
    print(t.get_text())

In [ ]:
soup.find_all("a")